In [1]:
import pandas as pd, numpy as np
import os
import pyreadr
from dsc.dsc_io import load_rds
cwd = os.path.expanduser("~/GIT/cnv-gene-mapping/data")
folder = "deletion"

In [2]:
fisher = pd.read_csv(f"{cwd}/{folder}/deletion.genes.block1.fisher.gz", header = 0, sep = "\t")

In [3]:
index = pd.read_csv(f"{cwd}/{folder}/deletion.block1.index.csv", header = None, sep = "\t", names = ["id1", "id2"])

In [5]:
index.head()

,id1,id2
0,0,7
1,8,17
2,18,22
3,23,36
4,37,44


In [6]:
fisher_in_block = list()
gene_in_block = list()
d_c = list()
d_nc = list()
nd_c = list()
nd_nc = list()
for idx, item in index.iterrows():
    fisher_in_block.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[0], item[1]+1)]].reset_index()["p"].tolist())
    gene_in_block.append([f"gene_{i+1}" for i in range(item[0], item[1]+1)])
    d_c.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[0], item[1]+1)]].reset_index()["d_c"].tolist())
    d_nc.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[0], item[1]+1)]].reset_index()["d_nc"].tolist())
    nd_c.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[0], item[1]+1)]].reset_index()["nd_c"].tolist())
    nd_nc.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[0], item[1]+1)]].reset_index()["nd_nc"].tolist())

In [8]:
index.shape

(528, 2)

In [11]:
logit_pymc3 = pd.DataFrame()
for idx, item in index.iterrows():
    logit = pyreadr.read_r(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block_{item[0]}_{item[1]}.logit.rds")
    pymc3 = pd.read_csv(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block_{item[0]}_{item[1]}.pymc3.gz", sep = "\t")
    pymc3["block"] = f"block{idx+1}"
    susie = load_rds(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block_{item[0]}_{item[1]}.SuSiE.L_1.prior_0p005.susie.rds")["pip"]
    varbvs = load_rds(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block_{item[0]}_{item[1]}.varbvs.rds")["pip"]
    res = pd.concat([logit[None][["p1"]], pymc3[["inclusion_probability", "block"]]], axis = 1)
    res["susie"] = susie
    res["varbvs"] = varbvs
    res["fisher"] = fisher_in_block[idx]
    res["gene"] = gene_in_block[idx]
    res["d_c"] = d_c[idx]
    res["d_nc"] = d_nc[idx]
    res["nd_c"] = nd_c[idx]
    res["nd_nc"] = nd_nc[idx]
    res = res.rename(columns = {"p1": "logit", "inclusion_probability": "pymc3"})
    logit_pymc3 = pd.concat([logit_pymc3, res])

In [12]:
logit_pymc3["logit"] = [np.round(x, 6) for x in logit_pymc3["logit"]]
logit_pymc3["pymc3"] = [np.round(x, 6) for x in logit_pymc3["pymc3"]]
logit_pymc3["susie"] = [np.round(x, 6) for x in logit_pymc3["susie"]]
logit_pymc3["varbvs"] = [np.round(x, 6) for x in logit_pymc3["varbvs"]]
logit_pymc3["fisher"] = [np.round(x, 6) for x in logit_pymc3["fisher"]]

In [13]:
logit_pymc3 = logit_pymc3[["block", "gene", "fisher", "d_c", "d_nc", "nd_c", "nd_nc", "pymc3", "logit", "susie", "varbvs"]]

In [15]:
logit_pymc3.head(20)

,block,gene,fisher,d_c,d_nc,nd_c,nd_nc,pymc3,logit,susie,varbvs
0,block1,gene_1,0.460791,1,1421,0,1664,0.0645,0.125,0.125,0.100334
1,block1,gene_2,0.460791,1,1421,0,1664,0.0730,0.125,0.125,0.100334
2,block1,gene_3,0.460791,1,1421,0,1664,0.0645,0.125,0.125,0.100334
3,block1,gene_4,0.460791,1,1421,0,1664,0.0605,0.125,0.125,0.100334
4,block1,gene_5,0.460791,1,1421,0,1664,0.0620,0.125,0.125,0.100335
5,block1,gene_6,0.460791,1,1421,0,1664,0.0665,0.125,0.125,0.100335
6,block1,gene_7,0.460791,1,1421,0,1664,0.0740,0.125,0.125,0.100334
7,block1,gene_8,0.460791,1,1421,0,1664,0.0700,0.125,0.125,0.100334
0,block2,gene_9,0.460791,1,1421,0,1664,0.0675,0.100,0.100,0.090416
1,block2,gene_10,0.460791,1,1421,0,1664,0.0670,0.100,0.100,0.090416


In [16]:
logit_pymc3.to_csv(f"{cwd}/{folder}/Real_deletion_res.csv", sep = "\t", header = True, index = False)

In [17]:
logit_pymc3[logit_pymc3["fisher"] < 0.01]

,block,gene,fisher,d_c,d_nc,nd_c,nd_nc,pymc3,logit,susie,varbvs
6,block128,gene_567,0.009518,6,1416,0,1664,0.0975,0.042954,0.042875,0.078196
7,block128,gene_568,0.009518,6,1416,0,1664,0.1140,0.042954,0.042875,0.078201
8,block128,gene_569,0.009518,6,1416,0,1664,0.1120,0.042954,0.042875,0.078207
9,block128,gene_570,0.009518,6,1416,0,1664,0.1015,0.042954,0.042875,0.078212
10,block128,gene_571,0.009518,6,1416,0,1664,0.0975,0.042954,0.042875,0.078217
11,block128,gene_572,0.009518,6,1416,0,1664,0.0955,0.042954,0.042875,0.078221
12,block128,gene_573,0.009518,6,1416,0,1664,0.1105,0.042954,0.042875,0.078225
13,block128,gene_574,0.009518,6,1416,0,1664,0.0985,0.042954,0.042875,0.078228
14,block128,gene_575,0.009518,6,1416,0,1664,0.0995,0.042954,0.042875,0.078230
15,block128,gene_576,0.009518,6,1416,0,1664,0.1205,0.042954,0.042875,0.078231
